# benchmark

benchmark a few different embedding options on relevant datasets

## dataset ideas

* classifier, classes are spotify genres of songs in existing library
* classifier, boolean classes over tags in rekordbox

In [1]:
import pickle
import torch
import torchaudio
import time
from audiocraft.models import MusicGen
from audiocraft.data.audio import audio_write
from audiocraft.data.audio_utils import convert_audio
from audiocraft.modules.conditioners import ConditioningAttributes

objc[69536]: Class AVFFrameReceiver is implemented in both /Users/maxj/miniconda3/envs/audiocraft/lib/libavdevice.58.8.100.dylib (0x117ff0798) and /Users/maxj/miniconda3/envs/audiocraft/lib/python3.9/site-packages/av/.dylibs/libavdevice.59.7.100.dylib (0x1588fc778). One of the two will be used. Which one is undefined.
objc[69536]: Class AVFAudioReceiver is implemented in both /Users/maxj/miniconda3/envs/audiocraft/lib/libavdevice.58.8.100.dylib (0x117ff07e8) and /Users/maxj/miniconda3/envs/audiocraft/lib/python3.9/site-packages/av/.dylibs/libavdevice.59.7.100.dylib (0x1588fc7c8). One of the two will be used. Which one is undefined.


In [2]:
model = MusicGen.get_pretrained('facebook/musicgen-medium')

/Users/maxj/miniconda3/envs/audiocraft/lib/python3.9/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [3]:
def compute_file_embedding(filename):
    return _embed_codes(filename, _load_input(filename))

def _load_input(filename):
    t1 = time.perf_counter()
    wav, sr = torchaudio.load(filename)
    duration = wav.shape[1] / sr
    if duration > 25 * 60:
        raise ValueError(f"track is too long: {duration}s")
    wav = convert_audio(wav, sr, model.sample_rate, model.audio_channels)
    wav = wav.unsqueeze(0)
    t2 = time.perf_counter()
    print(filename, "load", t2 - t1)
    with torch.no_grad():
        encoded, _ = model.compression_model.encode(wav)
    t3 = time.perf_counter()
    print(filename, "encode", t3-t2)
    return encoded
    
def _embed_codes(filename, encoded):
    t1 = time.perf_counter()
    context_limit = model.max_duration * model.frame_rate - 1
    emb = model.lm.compute_sequence_embedding(encoded[0], context_limit)
    t2 = time.perf_counter()
    print(filename, "embed", t2-t1)
    return emb

In [4]:
import pandas as pd
import sqlite3

con = sqlite3.connect("plaintext.db")
df_tracks = pd.read_sql_query("SELECT `ID` as id, `FolderPath` as path FROM djmdContent WHERE `rb_local_deleted` = 0;", con)

df_tracks

,id,path
0,178582527,/Users/maxj/Music/PioneerDJ/Sampler/OSC_SAMPLE...
1,35341928,/Users/maxj/Music/PioneerDJ/Sampler/OSC_SAMPLE...
2,196758694,/Users/maxj/Music/PioneerDJ/Sampler/OSC_SAMPLE...
3,96986769,/Users/maxj/Music/PioneerDJ/Sampler/OSC_SAMPLE...
4,78147589,/Users/maxj/Music/PioneerDJ/Demo Tracks/Demo T...
...,...,...
1255,233763711,/Users/maxj/Music/Rips/Spotify/AIFF/Nikitch - ...
1256,238880670,/Users/maxj/Music/Rips/Spotify/AIFF/KAM-BU - E...
1257,255506951,/Users/maxj/Music/Rips/Spotify/AIFF/RAPRAVE - ...
1258,108746388,/Users/maxj/Music/Rips/Spotify/AIFF/NxxxxxS - ...


In [5]:
try:
    with open('embeddings.pickle', 'rb') as f:
        embs = pickle.load(f)
        print('cached embeddings', len(embs))
except FileNotFoundError:
    print('empty embeddings')
    embs = {}

cached embeddings 1249


In [6]:
import pickle
import tempfile
import os

def save(embeddings):
    # Create a temporary file
    with tempfile.NamedTemporaryFile(delete=False) as temp_file:
        temp_filename = temp_file.name

        # Dump the data into the temporary file
        with open(temp_filename, 'wb') as f:
            pickle.dump(embs, f)

        # Rename the temporary file to the desired filename
        os.rename(temp_filename, "embeddings.pickle")


In [7]:
import sys
import traceback

for p in df_tracks["path"]:
    if p in embs:
        print("cached", p)
        continue
    if p.startswith('/Users/maxj/Music/PioneerDJ/Recording') or p.startswith('/Users/maxj/Music/Sets'):
        print("skip", p)
        continue
    try:
        embs[p] = compute_file_embedding(p)
    except:
        traceback.print_exception(*sys.exc_info())
        continue
    t1 = time.perf_counter()
    save(embs)
    print(p, "pickle", time.perf_counter()-t1)

cached /Users/maxj/Music/PioneerDJ/Sampler/OSC_SAMPLER/PRESET ONESHOT/NOISE.wav
cached /Users/maxj/Music/PioneerDJ/Sampler/OSC_SAMPLER/PRESET ONESHOT/SINEWAVE.wav
cached /Users/maxj/Music/PioneerDJ/Sampler/OSC_SAMPLER/PRESET ONESHOT/SIREN.wav
cached /Users/maxj/Music/PioneerDJ/Sampler/OSC_SAMPLER/PRESET ONESHOT/HORN.wav
cached /Users/maxj/Music/PioneerDJ/Demo Tracks/Demo Track 1.mp3
cached /Users/maxj/Music/PioneerDJ/Demo Tracks/Demo Track 2.mp3
cached /Users/maxj/Music/PioneerDJ/Sampler/MERGE FX/MergeFX Sample Sound 104.wav
cached /Users/maxj/Music/PioneerDJ/Sampler/MERGE FX/MergeFX Sample Sound 304.wav
cached /Users/maxj/Music/PioneerDJ/Sampler/MERGE FX/MergeFX Sample Sound 305.wav
cached /Users/maxj/Music/PioneerDJ/Sampler/MERGE FX/MergeFX Sample Sound 301.wav
cached /Users/maxj/Music/PioneerDJ/Sampler/MERGE FX/MergeFX Sample Sound 103.wav
cached /Users/maxj/Music/PioneerDJ/Sampler/MERGE FX/MergeFX Sample Sound 102.wav
cached /Users/maxj/Music/PioneerDJ/Sampler/MERGE FX/MergeFX Samp

Traceback (most recent call last):
  File "/var/folders/mh/p3fbt5x10rnb74s6sh1dql1c0000gn/T/ipykernel_69536/4082081347.py", line 12, in <module>
    embs[p] = compute_file_embedding(p)
  File "/var/folders/mh/p3fbt5x10rnb74s6sh1dql1c0000gn/T/ipykernel_69536/1900485787.py", line 2, in compute_file_embedding
    return _embed_codes(filename, _load_input(filename))
  File "/var/folders/mh/p3fbt5x10rnb74s6sh1dql1c0000gn/T/ipykernel_69536/1900485787.py", line 6, in _load_input
    wav, sr = torchaudio.load(filename)
  File "/Users/maxj/miniconda3/envs/audiocraft/lib/python3.9/site-packages/torchaudio/_backend/utils.py", line 203, in load
    return backend.load(uri, frame_offset, num_frames, normalize, channels_first, format, buffer_size)
  File "/Users/maxj/miniconda3/envs/audiocraft/lib/python3.9/site-packages/torchaudio/_backend/ffmpeg.py", line 334, in load
    return load_audio(os.path.normpath(uri), frame_offset, num_frames, normalize, channels_first, format)
  File "/Users/maxj/minic

In [8]:
 wav

NameError: name 'wav' is not defined